In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
group_dir_ = '../data/groups/low'
folder = 'all (user)'
file_name = 'normalized_to_rating_filter_track_5_user_100.pkl'

In [2]:
predictions = pd.read_pickle(os.path.join(dir_, folder, 'topN_pred_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
26,0,9,4.75,15
37,0,15,2.50,3
45,0,22,3.50,6
64,0,28,2.75,4
75,0,33,0.25,1


In [3]:
predictions[:5]

,uid,tid,rating
0,0,9,0.847839
1,0,15,0.594322
2,0,22,0.446187
3,0,28,0.721104
4,0,33,0.597911


In [4]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)
    print(len(groups[0]))

2
3
4
5
6
7
8


In [5]:
num_user = len(test['uid'].unique())
num_user

953

In [6]:
# Personal Recommendation
n=20
top_n_lists = []

for i in tqdm(range(num_user)):
    prediction = predictions[predictions['uid'] == i]
    prediction = prediction.sort_values(by=['rating'],  ascending=False)
    prediction = prediction[:n]
#     print(prediction)
    top_n_list = []
    for _, row in prediction.iterrows():
        top_n_list.append(row[1])
#     print(top_n_list)
    top_n_lists.append(top_n_list)

In [7]:
evaluation = []
satisfication = 0
for i in tqdm(range(num_user)):
    top_n_list = top_n_lists[i]
    precision = 0
    for j in top_n_list:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision += 1
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/num_user)


0.2091290661070307


In [8]:
precision_4 = []
precision_20 = []
precision_50 = []

recall_4 = []
recall_20 = []
recall_50 = []

In [9]:
# Precision and Recall
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_precisions = []
    avg_recalls = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        recalls =[]
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            
            for k in range(len(group)):
                high_rating = 0
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        
                precision = high_rating / top_n_size
                recall = high_rating / len(test[test['uid'] == uid])
                precisions.append(precision)
                recalls.append(recall)
                
        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
        
        avg_recall = 0
        for recall in recalls:
            avg_recall += recall
        avg_recall /= len(recalls)
        avg_recalls.append(avg_recall)
        
    if top_n_size == 4:
        precision_4.append(avg_precisions)
    elif top_n_size == 20:
        precision_20.append(avg_precisions)
    elif top_n_size == 50:
        precision_50.append(avg_precisions)
        
    if top_n_size == 4:
        recall_4.append(avg_recalls)
    elif top_n_size == 20:
        recall_20.append(avg_recalls)
    elif top_n_size == 50:
        recall_50.append(avg_recalls)


In [10]:
for i in precision_4:
    print(i)

for i in precision_20:
    print(i)

for i in precision_50:
    print(i)

for i in recall_4:
    print(i)

for i in recall_20:
    print(i)

for i in recall_50:
    print(i)

[0.2325, 0.2037037037037037, 0.21394230769230768, 0.19666666666666666, 0.17803030303030304, 0.20408163265306123, 0.15625]
[0.17500000000000007, 0.1607407407407408, 0.14903846153846156, 0.15200000000000005, 0.13863636363636364, 0.12959183673469385, 0.1125]
[0.14239999999999983, 0.12296296296296284, 0.12403846153846146, 0.12079999999999996, 0.12363636363636361, 0.10612244897959182, 0.10100000000000005]
[0.0016107618828370273, 0.0014188390578868829, 0.0012222603147108704, 0.0009885525641906102, 0.000945259408165759, 0.0010242794021052323, 0.0006997208490741338]
[0.00586532652454818, 0.005381783463040416, 0.004132258925233386, 0.004124310204326865, 0.004143571016043072, 0.0034184357357550967, 0.003013659473850876]
[0.012106425107514274, 0.01096404252387334, 0.008887108490122336, 0.00877519525261372, 0.009693802642061624, 0.0076159998824398166, 0.006614378804518128]


In [11]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    for i in range(len(r)):
        r[i] = 2**r[i] -1
    
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, r_max, k, method=0):
    dcg_max = dcg_at_k(r_max, k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [12]:
nDCG_4 = []
nDCG_20 = []
nDCG_50 = []

In [13]:
# Precision and Recall
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_nDCGs = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        nDCGs = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            
            for k in range(len(group)):
                truth_rating = []
                uid = group[k]
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    
                    if len(t) > 0:
                        truth_rating.append(t.iloc[0]['rating'])
                    else:
                        truth_rating.append(0)
                max_rating = test[test['uid']==uid].sort_values(by=['rating'],  ascending=False)['rating'].values[:top_n_size]
                nDCG = ndcg_at_k(truth_rating, max_rating, top_n_size, method=1)
                nDCGs.append(nDCG)                
        avg_nDCG = 0
        for nDCG in nDCGs:
            avg_nDCG += nDCG
        avg_nDCG /= len(nDCGs)
        avg_nDCGs.append(avg_nDCG)
        
    if top_n_size == 4:
        nDCG_4.append(avg_nDCGs)
    elif top_n_size == 20:
        nDCG_20.append(avg_nDCGs)
    elif top_n_size == 50:
        nDCG_50.append(avg_nDCGs)

In [14]:
for i in nDCG_4:
    print(i)

for i in nDCG_20:
    print(i)

for i in nDCG_50:
    print(i)

[0.11701630631855647, 0.09739212899518522, 0.10973716734661942, 0.09999836744164903, 0.08933548517545196, 0.11931350402953735, 0.07994072709913817]
[0.08501879558531752, 0.0754293653598881, 0.07693905160680974, 0.07614016072733763, 0.06642266098765016, 0.06972734955439261, 0.05087707074090371]
[0.06896163265821924, 0.06143771110153302, 0.06170434727910115, 0.060568545422737013, 0.05708832954080001, 0.05258988868920109, 0.04494685085801265]
